In [1]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/ryanchung403/dataset/main/Housing_Dataset_Sample.csv")
df.head(n=6)
df.isnull().sum()

Avg. Area Income                0
Avg. Area House Age             0
Avg. Area Number of Rooms       0
Avg. Area Number of Bedrooms    0
Area Population                 0
Price                           0
Address                         0
dtype: int64

In [3]:
X = df.iloc[:,:5]
y = df['Price']
X

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population
0,79545.458574,5.682861,7.009188,4.09,23086.800503
1,79248.642455,6.002900,6.730821,3.09,40173.072174
2,61287.067179,5.865890,8.512727,5.13,36882.159400
3,63345.240046,7.188236,5.586729,3.26,34310.242831
4,59982.197226,5.040555,7.839388,4.23,26354.109472
...,...,...,...,...,...
4995,60567.944140,7.830362,6.137356,3.46,22837.361035
4996,78491.275435,6.999135,6.576763,4.02,25616.115489
4997,63390.686886,7.250591,4.805081,2.13,33266.145490
4998,68001.331235,5.534388,7.130144,5.44,42625.620156


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=54)

In [5]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)
predictions =  reg.predict(X_test)

from sklearn.metrics import r2_score
r2_score(y_test, predictions)

0.921660486570713

# 更改test_size

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=54)
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)
predictions =  reg.predict(X_test)

from sklearn.metrics import r2_score
r2_score(y_test, predictions)

0.9218690199428807

將test_size改為0.2之後，r2 score大約上升了0.002，參考資料中指出，通常test data的大小設為0.2時，訓練效果會最好。
推測因為投入了更多資料讓模型訓練，模型學習的較好，因此r2 score上升，不過也不是訓練資料越多越好，有可能導致overfitting，影響模型預測效果。
參考資料:
https://medium.com/data-science-365/how-large-does-the-test-data-set-need-to-be-7181922d5dda，

# 調整係數

In [7]:
#找出linearregression係數
reg.get_params().keys()

dict_keys(['copy_X', 'fit_intercept', 'n_jobs', 'positive'])

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
premodel = LinearRegression()
parameters={
    'fit_intercept':[True,False], 'copy_X':[True, False],'positive':[True, False]
}

upgradepara=GridSearchCV(premodel,parameters,cv=3,scoring='r2')
upgradepara.fit(X_train, y_train)
print("最好的參數",upgradepara.best_params_)


最好的參數 {'copy_X': True, 'fit_intercept': True, 'positive': False}


In [9]:
#upgrademodel=LinearRegression('copy_X'= True, 'fit_intercept': True, 'n_jobs': None, 'positive': True)
predictions_test =  upgradepara.predict(X_test)

from sklearn.metrics import r2_score
r2_score(y_test, predictions_test)

0.9218690199428807

使用GridSearchCV找出linear regression模型中最合適的參數，Gridsearch為網格搜索，在所有的參數中遍歷每一種可能性，CV作交叉驗證，最後找出表現最好的參數。
不過實作結果，在參數調整前後，r2 score並沒有改變。

# 更改模型_Decision Tree

In [11]:
from sklearn.tree import DecisionTreeRegressor
tree=DecisionTreeRegressor()
tree.fit(X_train,y_train)
y_pred = tree.predict(X_test)
r2_score(y_test, y_pred)


0.7571401488002616

將模型改為Decision Tree，r2 score為0.75

In [12]:
from sklearn.tree import DecisionTreeRegressor
param_grid=[
    {'max_depth':range(5,50,5),
     'min_samples_split':range(10,100,1)}
]
premodel=DecisionTreeRegressor(random_state=0)
model_dt=GridSearchCV(premodel,param_grid,cv=3,scoring='r2')
model_dt.fit(X_train,y_train)
print("最好的模型",model_dt.best_estimator_)
print("最好的參數",model_dt.best_params_)

最好的模型 DecisionTreeRegressor(max_depth=10, min_samples_split=10, random_state=0)
最好的參數 {'max_depth': 10, 'min_samples_split': 10}


In [13]:
upgrade_dt=DecisionTreeRegressor(max_depth=10, min_samples_split=14, random_state=0)
upgrade_dt.fit(X_train,y_train)
predictions_dt =  upgrade_dt.predict(X_test)

from sklearn.metrics import r2_score
r2_score(y_test, predictions_dt)

0.8011005582755315

進一步調整參數。
使用GridsearchCV找出最好的參數之後，r2 score上升至0.8，但仍比原來的0.92低。